In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, when, col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import matplotlib.pyplot as plt

In [2]:
findspark.init()
spark = SparkSession.builder.appName("bank_info").getOrCreate()

23/11/22 21:43:37 WARN Utils: Your hostname, admins-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.4.50 instead (on interface en0)
23/11/22 21:43:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/22 21:43:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path = "Loan_default.csv"

In [4]:
bank_df = spark.read.option("header",'True').option('delimiter', ',').csv(path)
bank_df.show()

+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|    LoanID|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|
+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|I38PQUQS96| 56| 85994|     50587|        520|            80|             4|       15.23|      36|    0.44| Bachelor's|     Full-time|     Divorced|        Yes|          Yes|      Other|        Yes|      0|
|HPSK72WA7R| 69| 50432|    124440|        458|            15|             1|        4.81|      60|    0.68|   Master's|     Full-time|      Married|         No|           N

In [5]:
num_rows = bank_df.count()
num_columns = bank_df.columns
print("Number of rows:", num_rows)
print("Number of columns:", len(num_columns))

Number of rows: 255347
Number of columns: 18


In [6]:
bank_df.printSchema()

root
 |-- LoanID: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- LoanAmount: string (nullable = true)
 |-- CreditScore: string (nullable = true)
 |-- MonthsEmployed: string (nullable = true)
 |-- NumCreditLines: string (nullable = true)
 |-- InterestRate: string (nullable = true)
 |-- LoanTerm: string (nullable = true)
 |-- DTIRatio: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EmploymentType: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- HasMortgage: string (nullable = true)
 |-- HasDependents: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- HasCoSigner: string (nullable = true)
 |-- Default: string (nullable = true)



In [7]:
# Create a new DataFrame with the correct colums as a float datatype. 
columns_to_cast = ["Age", "Income", "LoanAmount", "CreditScore", "MonthsEmployed", "NumCreditLines", "InterestRate", "LoanTerm", "DTIRatio","Default"]

for i in columns_to_cast:
    bank_df = bank_df.withColumn(i, col(i).cast('float'))

In [8]:
bank_df.printSchema()

root
 |-- LoanID: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- Income: float (nullable = true)
 |-- LoanAmount: float (nullable = true)
 |-- CreditScore: float (nullable = true)
 |-- MonthsEmployed: float (nullable = true)
 |-- NumCreditLines: float (nullable = true)
 |-- InterestRate: float (nullable = true)
 |-- LoanTerm: float (nullable = true)
 |-- DTIRatio: float (nullable = true)
 |-- Education: string (nullable = true)
 |-- EmploymentType: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- HasMortgage: string (nullable = true)
 |-- HasDependents: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- HasCoSigner: string (nullable = true)
 |-- Default: float (nullable = true)



In [9]:
bank_df.createOrReplaceTempView("bank_table")

In [10]:
# Create a table showing the average Debt-To-Income Ratio and Default
query = """
SELECT COUNT(*) AS TotalRecords, AVG(DTIRatio) AS AVGDTIRatio, AVG(Default) AS AVGDefault
FROM bank_table
"""
spark.sql(query).show()

+------------+------------------+-------------------+
|TotalRecords|       AVGDTIRatio|         AVGDefault|
+------------+------------------+-------------------+
|      255347|0.5002120635652542|0.11612824901017048|
+------------+------------------+-------------------+



In [11]:
# SQL query to select ages over 55 and their interest rate
query = """
SELECT Age, InterestRate
FROM bank_table
WHERE Age > 55
"""


spark.sql(query).show()

+----+------------+
| Age|InterestRate|
+----+------------+
|56.0|       15.23|
|69.0|        4.81|
|60.0|        6.51|
|56.0|        8.15|
|57.0|        5.63|
|61.0|       15.58|
|61.0|       23.91|
|66.0|        6.67|
|59.0|       16.11|
|64.0|        9.23|
|68.0|        2.96|
|61.0|        20.0|
|69.0|       13.82|
|68.0|       18.84|
|56.0|         8.2|
|67.0|        3.74|
|64.0|       23.41|
|61.0|        22.8|
|64.0|       18.01|
|67.0|        4.14|
+----+------------+
only showing top 20 rows



In [12]:
# Average Interest Rate for people over 55
query = """
SELECT AVG(InterestRate) AS AvgInterestRateOver55
FROM bank_table
WHERE Age > 55
"""
spark.sql(query).show()

+---------------------+
|AvgInterestRateOver55|
+---------------------+
|    13.48866078421842|
+---------------------+



In [13]:
# Average Interest Rate for people under 55
query = """
SELECT AVG(InterestRate) AS AvgInterestRateUnder55
FROM bank_table
WHERE Age < 55
"""
spark.sql(query).show()

+----------------------+
|AvgInterestRateUnder55|
+----------------------+
|     13.49782966915141|
+----------------------+



In [14]:
# SQL query to analyze the impact of interest rates on default rates
query = """
SELECT InterestRate, AVG(InterestRate) AS AvgInterestRate, AVG(Default) AS AvgDefaultRate
FROM bank_table
GROUP BY InterestRate
ORDER BY AvgDefaultRate DESC
"""

# Execute the query and show the results
spark.sql(query).show()

+------------+------------------+-------------------+
|InterestRate|   AvgInterestRate|     AvgDefaultRate|
+------------+------------------+-------------------+
|       21.28|21.280000686645508| 0.3008849557522124|
|       24.42|24.420000076293945| 0.2966101694915254|
|       24.69|24.690000534057617|  0.288135593220339|
|       22.25|             22.25|0.28440366972477066|
|       24.04|24.040000915527344|0.27450980392156865|
|       24.77|24.770000457763672|0.27380952380952384|
|       20.44|20.440000534057617|0.26851851851851855|
|       23.62|  23.6200008392334|0.26851851851851855|
|       21.02|21.020000457763672|0.26605504587155965|
|       22.86|22.860000610351562|0.26548672566371684|
|       23.83|23.829999923706055|0.25925925925925924|
|       24.39|24.389999389648438|0.25862068965517243|
|       24.78|24.780000686645508| 0.2571428571428571|
|       22.37|  22.3700008392334|               0.25|
|       23.55|23.549999237060547|               0.25|
|        18.2|18.20000076293

In [15]:
# Number of Credit Lines VS Default
query = """
SELECT NumCreditLines, AVG(Default) AS AvgDefaultRate
FROM bank_table
GROUP BY NumCreditLines
ORDER BY AvgDefaultRate DESC
"""

# Execute the query and show the results
spark.sql(query).show()

+--------------+-------------------+
|NumCreditLines|     AvgDefaultRate|
+--------------+-------------------+
|           4.0|0.12942392956179793|
|           3.0|0.11924679637810572|
|           2.0|0.11058786839232808|
|           1.0|0.10523334487207729|
+--------------+-------------------+



In [16]:
# Loan Term VS Default
query = """
SELECT LoanTerm, AVG(Default) AS AvgDefaultRate
FROM bank_table
GROUP BY LoanTerm
ORDER BY AvgDefaultRate DESC
"""

# Execute the query and show the results
spark.sql(query).show()

+--------+-------------------+
|LoanTerm|     AvgDefaultRate|
+--------+-------------------+
|    60.0|0.11696055049458498|
|    12.0|0.11617638401004769|
|    24.0|0.11607755494128487|
|    48.0|0.11574092170582027|
|    36.0| 0.1156851608859991|
+--------+-------------------+



In [17]:
# Converting PySpark DataFrame to Pandas DataFrame
df = spark.sql("SELECT * FROM bank_table")
bank_df = df.toPandas()

In [18]:
spark.stop()

In [19]:
default_records_df = pd.DataFrame(bank_df[(bank_df['Default']==1)])
non_default_records_df = pd.DataFrame(bank_df[(bank_df['Default']!=1)])

avg_default_dti = default_records_df['DTIRatio'].mean().__round__(5)
avg_non_default_dti = non_default_records_df['DTIRatio'].mean().__round__(5)

default_credit_lines = default_records_df['NumCreditLines'].mean().__round__(2)
non_default_credit_lines = non_default_records_df['NumCreditLines'].mean().__round__(2)

default_avg_interest = default_records_df['InterestRate'].mean().__round__(3)
non_default_avg_interest = non_default_records_df['InterestRate'].mean().__round__(3)

default_avg_income = default_records_df['Income'].mean().__round__(2)
non_default_avg_income = non_default_records_df['Income'].mean().__round__(2)


print(f'''Of the records for which the loan was defaulted, the average debt to income ratio is: {avg_default_dti}.'''),
print(f'''Of the records for which the loan was defaulted, the average interest rate is: {default_avg_interest}%.'''),
print(f'''Of the records for which the loan was defaulted, the average number of credit lines is: {default_credit_lines}.'''),
print(f'''Of the records for which the loan was defaulted, the average income is: ${default_avg_income}.'''),

print("  "),

print(f'''Of the records for which the loan did NOT default, the average debt to income ratio is: {avg_non_default_dti}.'''),
print(f'''Of the records for which the loan did NOT default the average interest rate is: {non_default_avg_interest}%.'''),
print(f'''Of the records for which the loan did NOT default, the average number of credit lines is: {non_default_credit_lines}.'''),
print(f'''Of the records for which the loan did NOT default, the average income is: ${non_default_avg_income}.''')

Of the records for which the loan was defaulted, the average debt to income ratio is: 0.512470006942749.
Of the records for which the loan was defaulted, the average interest rate is: 15.895999908447266%.
Of the records for which the loan was defaulted, the average number of credit lines is: 2.5899999141693115.
Of the records for which the loan was defaulted, the average income is: $71844.71875.
  
Of the records for which the loan did NOT default, the average debt to income ratio is: 0.4986000061035156.
Of the records for which the loan did NOT default the average interest rate is: 13.177000045776367%.
Of the records for which the loan did NOT default, the average number of credit lines is: 2.490000009536743.
Of the records for which the loan did NOT default, the average income is: $83899.15625.


### Machine Learning/Deep Learning

In [20]:
# Adding an index column (#)
bank_df.insert(0,'#',range(len(bank_df)))
bank_df.head()

,#,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,0,I38PQUQS96,56.0,85994.0,50587.0,520.0,80.0,4.0,15.23,36.0,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0.0
1,1,HPSK72WA7R,69.0,50432.0,124440.0,458.0,15.0,1.0,4.81,60.0,0.68,Master's,Full-time,Married,No,No,Other,Yes,0.0
2,2,C1OZ6DPJ8Y,46.0,84208.0,129188.0,451.0,26.0,3.0,21.17,24.0,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1.0
3,3,V2KKSFM3UN,32.0,31713.0,44799.0,743.0,0.0,3.0,7.07,24.0,0.23,High School,Full-time,Married,No,No,Business,No,0.0
4,4,EY08JDHTZP,60.0,20437.0,9139.0,633.0,8.0,4.0,6.51,48.0,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0.0


In [21]:
# Drop the non-beneficial ID columns
bank_df = bank_df.drop(columns=['Education','EmploymentType','MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner'])
bank_df

,#,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Default
0,0,I38PQUQS96,56.0,85994.0,50587.0,520.0,80.0,4.0,15.23,36.0,0.44,0.0
1,1,HPSK72WA7R,69.0,50432.0,124440.0,458.0,15.0,1.0,4.81,60.0,0.68,0.0
2,2,C1OZ6DPJ8Y,46.0,84208.0,129188.0,451.0,26.0,3.0,21.17,24.0,0.31,1.0
3,3,V2KKSFM3UN,32.0,31713.0,44799.0,743.0,0.0,3.0,7.07,24.0,0.23,0.0
4,4,EY08JDHTZP,60.0,20437.0,9139.0,633.0,8.0,4.0,6.51,48.0,0.73,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
255342,255342,8C6S86ESGC,19.0,37979.0,210682.0,541.0,109.0,4.0,14.11,12.0,0.85,0.0
255343,255343,98R4KDHNND,32.0,51953.0,189899.0,511.0,14.0,2.0,11.55,24.0,0.21,1.0
255344,255344,XQK1UUUNGP,56.0,84820.0,208294.0,597.0,70.0,3.0,5.29,60.0,0.50,0.0
255345,255345,JAO28CPL4H,42.0,85109.0,60575.0,809.0,40.0,1.0,20.90,48.0,0.44,0.0
